## 38. Descriptor protocol (`__get__`, `__set__`, `__delete__`)

Descriptors let one object control attribute access of another. `property` and most ORMs are built on this. Gotcha: define them at *class* level, not inside `__init__`, else they act like normal attributes.

In [ ]:
class Positive:
    def __get__(self, obj, owner):
        return obj._value
    def __set__(self, obj, val):
        if val < 0: raise ValueError
        obj._value = val

class Account:
    balance = Positive()
    def __init__(self, start):
        self.balance = start

a = Account(10)
a.balance = 5
print(a.balance)

### Quick check

1. T / F `obj.attr` triggers `__get__` on the descriptor.

2. Writing `self.balance = -1` will:
  a. set value  b. raise ValueError

<details><summary>Answer key</summary>

1. **True**.
2. **b**.

</details>

## 39. Context managers: `with`, `__enter__`, `__exit__`

Context managers guarantee cleanup (closing files, locks) even on exceptions. Implement with the dunders or use `contextlib.contextmanager`.

In [ ]:
from contextlib import contextmanager
@contextmanager
def tag(name):
    print(f'<{name}>')
    yield
    print(f'</{name}>')

with tag('b'): print('bold')

### Quick check

1. `__exit__` receives how many exception args?
  a. 1  b. 3

2. T / F Returning True from `__exit__` suppresses the exception.

<details><summary>Answer key</summary>

1. **b** — exc_type, exc_val, traceback.
2. **True**.

</details>

## 40. Function and class decorators

Decorators wrap objects to add behaviour: memoization, logging, access control. Use `functools.wraps` to preserve metadata.

In [ ]:
from functools import wraps
def logger(fn):
    @wraps(fn)
    def inner(*a, **kw):
        print('calling', fn.__name__)
        return fn(*a, **kw)
    return inner

@logger
def add(a,b): return a+b

print(add(1,2))

### Quick check

1. T / F `@decorator` is executed at *import time*.

2. `wraps` copies:
  a. doc & name  b. nothing

<details><summary>Answer key</summary>

1. **True** — on function definition.
2. **a**.

</details>

## 41. Dynamic attribute hooks (`__getattr__`, `__getattribute__`)

`__getattr__` runs *after* normal lookup fails; good for lazy loading. `__getattribute__` intercepts **every** attribute access—use with care or risk infinite recursion.

In [ ]:
class Lazy:
    def __getattr__(self, item):
        print('loading', item)
        value = item.upper()
        setattr(self, item, value)
        return value

obj=Lazy()
print(obj.name)
print(obj.name)

### Quick check

1. First call prints 'loading', second?
  a. prints again  b. cached no print

2. T / F Misusing `__getattribute__` can break `super()`.

<details><summary>Answer key</summary>

1. **b** — attribute now cached.
2. **True**.

</details>

## 42. `__slots__` and memory optimisation

Adding `__slots__ = ('x', 'y')` removes per‑instance `__dict__`, saving memory in large homogeneous collections. You lose dynamic attribute assignment and some pickling flexibility.

In [ ]:
class Point:
    __slots__=('x','y')
    def __init__(self,x,y):
        self.x,self.y=x,y

p=Point(1,2)
print(p.x)

### Quick check

1. T / F Instances with `__slots__` have `__dict__` by default.

2. Trying `p.z=3` will:
  a. succeed  b. AttributeError

<details><summary>Answer key</summary>

1. **False**.
2. **b**.

</details>

## 43. `dataclasses` and `attrs`

`@dataclass` auto‑generates `__init__`, `__repr__`, comparison, and more. Use for plain‑data containers instead of verbose manual classes.

In [ ]:
from dataclasses import dataclass
@dataclass
class Point:
    x: int
    y: int

print(Point(1,2))

### Quick check

1. T / F `@dataclass(frozen=True)` makes instances hashable.

2. Default comparison order is:
  a. attribute definition order  b. alphabetical

<details><summary>Answer key</summary>

1. **True**.
2. **a**.

</details>

## 44. Abstract Base Classes & the `abc` module

ABCs declare required methods without implementation. Registering virtual subclasses allows duck typing with `isinstance(obj, MyABC)`.

In [ ]:
from abc import ABC, abstractmethod
class Serializer(ABC):
    @abstractmethod
    def dumps(self, obj): ...

class Json(Serializer):
    def dumps(self,obj):
        import json; return json.dumps(obj)

print(Json().dumps({'a':1}))

### Quick check

1. Instantiating `Serializer()` raises:
  a. TypeError  b. ValueError

2. T / F A subclass may skip overriding an abstract method.

<details><summary>Answer key</summary>

1. **a**.
2. **False** — must implement all.

</details>

## 45. Metaclass basics

A *metaclass* customises class creation. Rarely needed outside ORMs, plugins, or enforcing constraints. Use `__init_subclass__` for lighter hooks.

In [ ]:
class AutoName(type):
    def __new__(mcls,name,bases,ns):
        ns['__qualname__'] = name.lower()
        return super().__new__(mcls,name,bases,ns)

class Foo(metaclass=AutoName):
    pass
print(Foo.__qualname__)

### Quick check

1. T / F `type` is itself a metaclass.

2. Metaclasses intercept:
  a. instance creation  b. class creation

<details><summary>Answer key</summary>

1. **True**.
2. **b**.

</details>

## 46. Static type hints & `typing` basics

Type hints help linters (`mypy`, `pyright`) catch bugs without affecting runtime. Annotate variables and function signatures; keep comments in code for gradual adoption.

In [ ]:
def greet(name: str) -> str:
    return 'Hi ' + name

age: int = 42

### Quick check

1. T / F Type hints change runtime behaviour.

2. PEP governing typing is:
  a. PEP 484  b. PEP 8

<details><summary>Answer key</summary>

1. **False**.
2. **a**.

</details>

## 47. Generics & `Protocol`

Generic types parameterise containers (`list[int]`). `Protocol` enables structural typing: any object implementing the given attributes satisfies it, even without inheritance.

In [ ]:
from typing import TypeVar, Protocol, Iterable
T = TypeVar('T')

def first(it: Iterable[T]) -> T:
    return next(iter(it))

class Sized(Protocol):
    def __len__(self) -> int: ...

print(first(['a','b']))

### Quick check

1. T / F `Protocol` checks at runtime.

2. `list[int]` is available since:
  a. Python 3.7  b. 3.9

<details><summary>Answer key</summary>

1. **False** — static only.
2. **b** — PEP 585.

</details>

## 48. Structural pattern matching (`match` / `case`)

Introduced in Python 3.10, `match` provides switch‑like semantics with destructuring. Beware *fall‑through* does not exist; cases are checked top‑to‑bottom.

In [ ]:
def kind(value):
    match value:
        case int(): return 'int'
        case [x, y]: return 'pair'
        case _: return 'other'

print(kind([1,2]))

### Quick check

1. T / F `case _:` works like default.

2. Pattern `[x,y]` matches list of length:
  a. any  b. 2

<details><summary>Answer key</summary>

1. **True**.
2. **b**.

</details>

## 49. Enumerations with `enum.Enum`

Enums give named constants; iterate, compare identity, and serialize safely. Extending enums later breaks `auto()` numbering, so assign explicit values when stability matters.

In [ ]:
from enum import Enum, auto
class Color(Enum):
    RED = auto()
    GREEN = auto()

print(list(Color))

### Quick check

1. T / F Enum members are singleton objects.

2. `Color.RED == 1` is:
  a. True  b. False

<details><summary>Answer key</summary>

1. **True**.
2. **b** — compare to member, not value.

</details>

## 50. F‑strings & formatting mini‑language

F‑strings (`f'{var=:.2f}'`) are fastest and most readable. Format spec follows PEP 498 mini‑language: alignment, padding, numeric format. Common gotcha: braces inside need doubling `{{` `}}`.

In [ ]:
pi = 3.14159
print(f'{pi:.2f}')
name='Ada'
print(f'{name:>10}')

### Quick check

1. Which prints `0007` for x=7?
  a. `f'{x:4}'`  b. `f'{x:04}'`

2. T / F `f'{2+3=}'` prints `2+3=5`.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 51. Memory model, ref counting, GC

CPython uses reference counting plus a cyclic garbage collector. Objects free *as soon* as count drops to zero—so `with` is often sufficient resource cleanup. Circular refs rely on GC to break.

In [ ]:
import sys, gc
a = []
b = [a]
a.append(b)  # cycle
del a, b
print(gc.collect())  # runs GC

### Quick check

1. T / F `del a` always frees memory immediately.

2. Circular references are handled by:
  a. ref counting  b. cyclic GC

<details><summary>Answer key</summary>

1. **False** — only if no cycles.
2. **b**.

</details>

## 52. The Global Interpreter Lock (GIL)

In CPython, only one thread executes Python bytecode at a time. CPU‑bound tasks gain nothing from threads—use `multiprocessing` or native extensions instead. I/O‑bound workloads can still benefit via concurrency.

In [ ]:
import threading, time
def cpu():
    s = 0
    for _ in range(10_000_00): s+=1

t1=t2=threading.Thread(target=cpu)
start=time.time(); t1.start(); t2.start(); t1.join(); t2.join();
print('elapsed', time.time()-start)

### Quick check

1. T / F NumPy releases GIL inside heavy C loops.

2. Best for CPU‑bound concurrency:
  a. threads  b. processes

<details><summary>Answer key</summary>

1. **True** — many C extensions release it.
2. **b**.

</details>

## 53. Profiling & timing (`timeit`, `cProfile`)

Use `timeit` for micro‑benchmarks; `cProfile`/`snakeviz` to find hotspots. Avoid naive `time.time()` in loops due to noise.

In [ ]:
import timeit
print(timeit.timeit('sum(range(100))', number=1000))

### Quick check

1. `timeit` runs code:
  a. once  b. many times

2. T / F `cProfile` gives per‑function call counts.

<details><summary>Answer key</summary>

1. **b** — auto selects repeat count.
2. **True**.

</details>

## 54. Coroutine syntax: `async def` / `await`

Coroutines let you write non‑blocking code that *looks* sequential. Awaiting suspends so other tasks can run on the same thread. Mixing blocking I/O inside async breaks concurrency.

In [ ]:
import asyncio, time
async def hello():
    await asyncio.sleep(1)
    return 'hi'

print(asyncio.run(hello()))

### Quick check

1. `async def` returns:
  a. value  b. coroutine object

2. T / F You can `await` inside normal def.

<details><summary>Answer key</summary>

1. **b**.
2. **False** — only inside async def.

</details>

## 55. Creating & scheduling `asyncio` tasks

`asyncio.create_task()` schedules coroutines to run concurrently. Use `await task` for result; cancel long‑running tasks if no longer needed.

In [ ]:
import asyncio
async def worker(n):
    await asyncio.sleep(1)
    return n*n

async def main():
    tasks = [asyncio.create_task(worker(i)) for i in range(3)]
    print([await t for t in tasks])

asyncio.run(main())

### Quick check

1. T / F `create_task` starts running immediately.

2. Cancelling a task raises:
  a. CancelledError  b. TimeoutError

<details><summary>Answer key</summary>

1. **True** — scheduled on event loop tick.
2. **a**.

</details>